# Teaching GPT2 a sense of humor

This is simple experimental notebook for fine-tuning pretrained GPT2 model on jokes dataset. Let's see if it can learn to crack some jokes on it's own. 

For this purpose I will use pretrained models from huggingface [transformers repository](https://github.com/huggingface/transformers).

In [ ]:
!pip install transformers

In [1]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import numpy as np

import logging
logging.getLogger().setLevel(logging.CRITICAL)

import warnings
warnings.filterwarnings('ignore')

I1030 19:42:58.545788 140452527003392 file_utils.py:39] PyTorch version 1.1.0 available.
I1030 19:42:58.819280 140452527003392 modeling_xlnet.py:194] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


I will be using [GPT2LMHeadModel()](https://github.com/huggingface/transformers/blob/079bfb32fba4f2b39d344ca7af88d79a3ff27c7c/transformers/modeling_gpt2.py#L472) class, which is modified [GPT2Model()](https://github.com/huggingface/transformers/blob/079bfb32fba4f2b39d344ca7af88d79a3ff27c7c/transformers/modeling_gpt2.py#L320) for language modeling.

The only difference is added linear layer, which will transform the output embedding into logits (embedding size -> vocabulary size) which will be used for predicting the output words.

First I will do a small GPT2 model experiment to see if I have understood everything correctly and I can generate some text using the model.

In [2]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [3]:
# Function to first select topN tokens from the probability list and then based on the N word distribution
# select random token ID
def choose_from_top(probs, n=5):
    ind = np.argpartition(probs, -n)[-n:]
    top_prob = probs[ind]
    top_prob = top_prob / np.sum(top_prob) # Normalize
    choice = np.random.choice(n, 1, p = top_prob)
    token_id = ind[choice][0]
    return token_id

### First I prepare and tokenize the text which the model should start with continue itself. Then I run the model X iterations to add one token to the list in each iteration. 

In [6]:
cur_ids = torch.tensor(tokenizer.encode(" The Matrix is everywhere. It is all around us. Even now, in this very room. You can see it when you look out your window or when you turn on your television. You can feel it when you go to work... when you go to church... when you pay your taxes. It is the world that has been pulled over your eyes to blind you from the truth. ")).unsqueeze(0)

model.eval()
with torch.no_grad():
    
    for i in range(100):
        outputs = model(cur_ids, labels=cur_ids)
        loss, logits = outputs[:2]
        softmax_logits = torch.softmax(logits[0,-1], dim=0) #Take the first(only one) batch and the last predicted embedding
        next_token_id = choose_from_top(softmax_logits.numpy(), n=5) #Randomly(from the given probability distribution) choose the next word from the top n words
        cur_ids = torch.cat([cur_ids, torch.ones((1,1)).long() * next_token_id], dim = 1) # Add the last word

    output_list = list(cur_ids.squeeze().numpy())
    output_text = tokenizer.decode(output_list)
    print(output_text)

The Matrix is everywhere. It is all around us. Even now, in this very room. You can see it when you look out your window or when you turn on your television. You can feel it when you go to work... when you go to church... when you pay your taxes. It is the world that has been pulled over your eyes to blind you from the truth. It is the world that has been made to see. It is the world that is in control. It is the world that is out to destroy you. And it is the world that is the source of all evil, all darkness. And it is the world that is the source of all the evil. It is the world that is the source of all death and destruction. It is the world that is the source of everything we do in this life and all the evil we commit. It is the


### For safety will save this masterpiece in a notebook cell

*The Matrix is everywhere. It is all around us. Even now, in this very room. You can see it when you look out your window or when you turn on your television. You can feel it when you go to work... when you go to church... when you pay your taxes. It is the world that has been pulled over your eyes to blind you from the truth. It is the world that has been made into an object of fascination, and that you must confront. It is the world that is always there to keep you from being caught up in the truth. The Matrix is the place where you must fight to keep your eyes open and to keep yourself in your own mind... and to be a better man. And to be a better man.*

*I have a feeling that you have seen the truth. And you know what that means. I've seen it*

### Not bad. The model works and now we are ready to teach the GPT2 some sense of humor.

### The first step is to find and prepare the data.

In [4]:
import os
import json

In [5]:
from torch.utils.data import Dataset
from torch.utils.data import Dataset, DataLoader
import os
import json

class JokesDataset(Dataset):
    def __init__(self, jokes_dataset_path = 'jokes_data/'):
        super().__init__()

        reddit_jokes_path = os.path.join(jokes_dataset_path, 'reddit_jokes.json')

        with open(reddit_jokes_path) as f:
            data = json.load(f)

        self.joke_list = []
        self.end_of_text_token = "<|endoftext|>"

        for idx, joke_json in enumerate(data):
            joke_str = f"{self.end_of_text_token}START:{joke_json['title']} {joke_json['body']}{self.end_of_text_token}"
            self.joke_list.append(joke_str)
            
    def __len__(self):
        return len(self.joke_list)

    def __getitem__(self, item):
        return self.joke_list[item]


In [6]:
dataset = JokesDataset()
joke_loader = DataLoader(dataset, batch_size=1, shuffle=True)

In [7]:
for idx,joke in enumerate(joke_loader):
    print(joke)
    if idx>100:
        break

['<|endoftext|>START:The Bus Driver on Sesame Street One day a young husband gets a job as a bus driver on Sesame Street.  The morning of his first day on the job, his wife gives him a kiss goodbye and says, "Now I want to hear all about it when you come home, dear."\n\nSo he sets off and drives along Sesame Street.  He makes his first stop, and to his astonishment, on comes a super muscular, extremely-chiseled 6-year-old girl!  The girl says, "My name\'s Patricia...and I\'m LEAN!!" and takes her seat.  At the next stop, here is a little girl who is even more muscular than the first one...veins popping out, bulging biceps, the whole nine yards.  She steps on to the bus and says to the driver, "My name is Patricia...and I\'m MEAN!!!"  A little baffled by the look of his first two passengers, he continues on with his route.\n\nHe reaches the next stop, and feels relieved to see a normal-looking little boy, dressed in a sweater and tie.  He gets on the bus and says to the bus driver, "My 

In [8]:
BATCH_SIZE = 16
EPOCHS = 3
LEARNING_RATE = 1e-5
WARMUP_STEPS = 10000
from transformers import AdamW, WarmupLinearSchedule

device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
    


In [9]:
model = model.to(device)
model.train()
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
scheduler = WarmupLinearSchedule(optimizer, warmup_steps=WARMUP_STEPS, t_total = -1)
joke_count = 0
sum_loss = 0.0
batch_count = 0

for epoch in range(EPOCHS):
    
    print(f"EPOCH {epoch} started" + '=' * 30)
    
    for idx,joke in enumerate(joke_loader):

        joke_tens = torch.tensor(tokenizer.encode(joke[0])).unsqueeze(0).to(device)
        if joke_tens.size()[1] > 1024:
            continue
        
        outputs = model(joke_tens, labels=joke_tens)
        loss, logits = outputs[:2]                        
        loss.backward()
        sum_loss = sum_loss + loss.detach().data
                       
        joke_count = joke_count + 1
        if joke_count == BATCH_SIZE:
            joke_count = 0    
            batch_count += 1
            optimizer.step()
            scheduler.step()  
            model.zero_grad()
            
        if batch_count == 200:
            print(f"sum loss {sum_loss}")
            batch_count = 0
            sum_loss = 0.0

EPOCH 0 started==============================
sum loss 14599.1220703125
sum loss 13905.1640625
sum loss 12764.919921875
sum loss 11917.28125
sum loss 11409.8740234375
sum loss 11278.3173828125
sum loss 11163.931640625
sum loss 11020.099609375
sum loss 10894.953125
sum loss 10923.876953125
sum loss 10835.7998046875
sum loss 10742.3681640625
sum loss 10719.65234375
sum loss 10604.3525390625
sum loss 10535.2626953125
sum loss 10582.994140625
sum loss 10433.7724609375
sum loss 10426.5390625
sum loss 10396.8876953125
sum loss 10311.64453125
sum loss 10308.279296875
sum loss 10367.9599609375
sum loss 10254.341796875
sum loss 10248.7958984375
sum loss 10135.6455078125
sum loss 10099.4775390625
sum loss 10155.453125
sum loss 10134.83203125
sum loss 10073.927734375
sum loss 10087.70703125
sum loss 10085.1396484375
sum loss 10095.091796875
sum loss 9949.4599609375
sum loss 9867.2099609375
sum loss 9912.765625
sum loss 9897.4326171875
sum loss 9908.2099609375
sum loss 9810.314453125
sum loss 9842

In [10]:
model.eval()
with torch.no_grad():
    
    for joke_idx in range(1000):

        cur_ids = torch.tensor(tokenizer.encode("<|endoftext|>START:")).unsqueeze(0).to(device)
        
        for i in range(250):
            outputs = model(cur_ids, labels=cur_ids)
            loss, logits = outputs[:2]
            softmax_logits = torch.softmax(logits[0,-1], dim=0) #Take the first(only one) batch and the last predicted embedding
            if i < 2:
                n = 15
            else:
                n = 3
            next_token_id = choose_from_top(softmax_logits.to('cpu').numpy(), n=n) #Randomly(from the given probability distribution) choose the next word from the top n words
            cur_ids = torch.cat([cur_ids, torch.ones((1,1)).long().to(device) * next_token_id], dim = 1) # Add the last word

            if next_token_id in tokenizer.encode('<|endoftext|>'):
                break
            
        output_list = list(cur_ids.squeeze().to('cpu').numpy())
        output_text = tokenizer.decode(output_list)
        print(f"JOKE NR {joke_idx}: {output_text} \n")

JOKE NR 0: <|endoftext|>START:What did the pirate say when he saw the pirate's face? "You're a pirate!"<|endoftext|> 

JOKE NR 1: <|endoftext|>START:If you're a woman... You're a woman.<|endoftext|> 

JOKE NR 2: <|endoftext|>START:How do you make a girl cry? You put her on the bed.<|endoftext|> 

JOKE NR 3: <|endoftext|>START:How can a black man get a girlfriend? He's a black guy<|endoftext|> 

JOKE NR 4: <|endoftext|>START:So I got a job in a barber shop......and I'm like, "What the fuck is wrong with me?"

"I'm a fucking whore!"<|endoftext|> 

JOKE NR 5: <|endoftext|>START:What does a man and a dog have in common? They both have a dog.<|endoftext|> 

JOKE NR 6: <|endoftext|>START:How do you make your girlfriend laugh? Put her on a stool<|endoftext|> 

JOKE NR 7: <|endoftext|>START:I like my women like I like my coffee. Light, smooth and delicious.<|endoftext|> 

JOKE NR 8: <|endoftext|>START:There are two types of women: Those who can't stand the pressure of a man's presence, and the

JOKE NR 62: <|endoftext|>START:I don't like my women... They're all too fat.<|endoftext|> 

JOKE NR 63: <|endoftext|>START:I asked my girlfriend what I was going to do with my life after college... She said, "I'm going to go to college."<|endoftext|> 

JOKE NR 64: <|endoftext|>START:What did the chicken say to the cow? "Hey, I'm not hungry, I'm hungry"<|endoftext|> 

JOKE NR 65: <|endoftext|>START:What do you call a man with a big dick who can't stand a woman? A man with a big dick.<|endoftext|> 

JOKE NR 66: <|endoftext|>START:What's a pirate's favorite drink? A pirate's favourite drink.<|endoftext|> 

JOKE NR 67: <|endoftext|>START:I heard that the NSA is collecting phone records... I don't know why.<|endoftext|> 

JOKE NR 68: <|endoftext|>START:How do you get an Irishman to stop drinking? Put him in the car.<|endoftext|> 

JOKE NR 69: <|endoftext|>START:I had a really bad dream last night... I woke up in the middle of nowhere and I had no idea where I was going.<|endoftext|> 

JOKE 

JOKE NR 123: <|endoftext|>START:How did the dog get his name? He was a dog.

(I'm sorry)<|endoftext|> 

JOKE NR 124: <|endoftext|>START:I'm going to be the first to tell you that the world is a better place than it was before the AIDS pandemic. I'll tell you what it was like.<|endoftext|> 

JOKE NR 125: <|endoftext|>START:My girlfriend is so fat... She can't even stand up.<|endoftext|> 

JOKE NR 126: <|endoftext|>START:Did you hear about the man who was arrested for selling marijuana to prostitutes? He was arrested for possession and possession with intent to sell, but not a crime.<|endoftext|> 

JOKE NR 127: <|endoftext|>START:My life is like the Titanic Titanic. It's sinking fast.<|endoftext|> 

JOKE NR 128: <|endoftext|>START:What happened after the first black man died? He got a job at a factory.<|endoftext|> 

JOKE NR 129: <|endoftext|>START:What did the chicken say when it got to the other end of the table? "I'm not going to eat you!"<|endoftext|> 

JOKE NR 130: <|endoftext|>STAR

JOKE NR 182: <|endoftext|>START:What did the chicken say to the chicken? "You are so fat, you're a chicken!"<|endoftext|> 

JOKE NR 183: <|endoftext|>START:What do the Mexican and the Mexican have in common? They both have the same nose.<|endoftext|> 

JOKE NR 184: <|endoftext|>START:If you're a pedophile and you're a pedophiles lover... Then what do you get when you're a pedophile lover and you're a pedophiles lover's lover and you're pedophiles lover's lover? You get the pedophile lover's lover and you get the pedophiles lover.      

If you're a pedophiles lover and you're a pedophiles lover, you get the pedophiles lover and you get the pedophiles lover.

If you're a pedophiles lover and you're a pedophiles lover and you're a pedophiles lover, you get the pedophiles lover and you get the pedophiles lover.    

If you're a pedophiles lover and you're a pedophiles lover and you're a pedophiles lover and you're a pedophiles lover, you get the Pedophiles lover and you get the pedophiles

JOKE NR 235: <|endoftext|>START:The first time I met a woman......I was in a bar.<|endoftext|> 

JOKE NR 236: <|endoftext|>START:Did you hear about the guy who got a new job as an engineer? He got a job as an engineer.<|endoftext|> 

JOKE NR 237: <|endoftext|>START:What's the difference between a black person and a pedophile? You can't see a pedophile's dick.<|endoftext|> 

JOKE NR 238: <|endoftext|>START:What can a woman do when she's pregnant? You can't.<|endoftext|> 

JOKE NR 239: <|endoftext|>START:How do you know you're in the right place? You look in a mirror.<|endoftext|> 

JOKE NR 240: <|endoftext|>START:What do you call a Mexican with no teeth? A toothless<|endoftext|> 

JOKE NR 241: <|endoftext|>START:I have a problem with the word "joke." I don't like it when people call me a joke.<|endoftext|> 

JOKE NR 242: <|endoftext|>START:What's the difference between the best joke and the worst joke? The best joke is the worst.<|endoftext|> 

JOKE NR 243: <|endoftext|>START:My wife to

...

JOKE NR 772: <|endoftext|>START:I used to be a big fat guy. But I'm now a fat guy.<|endoftext|> 

JOKE NR 773: <|endoftext|>START:What's the difference between an apple and a grapefruit? An apple has a little more juice than a grapefruit.<|endoftext|> 

JOKE NR 774: <|endoftext|>START:If the world is a little bit like a chicken, why are the chickens so big? Because they're so small.<|endoftext|> 

JOKE NR 775: <|endoftext|>START:What do you call a Mexican who has been arrested? A Mexican with a gun<|endoftext|> 

JOKE NR 776: <|endoftext|>START:I hate the word 'nigger' because I hate the term 'nigger-in'.<|endoftext|> 

JOKE NR 777: <|endoftext|>START:I just bought my first car... I'm not sure what it is yet. I'm not sure how to tell if it's a new car, a car with a new engine, or a car with a new engine, or a car with a new engine, or a car with a new engine, or a car with a new engine, or a car with a new engine, or a car with a new engine, or a car with a new engine, or a car with a 

JOKE NR 827: <|endoftext|>START:I asked my girlfriend what her favourite kind of cheese was. She said she'd never heard a cheese with a cheese-like taste.<|endoftext|> 

JOKE NR 828: <|endoftext|>START:How do you make a girl cry? With a blowjob!<|endoftext|> 

JOKE NR 829: <|endoftext|>START:I had a really bad dream last night. I was in bed with a guy with a bad dream. He woke up and told me to go to bed and wake him up and I told him to go to sleep. I said "I'm not going to sleep with you, you're going to wake me up and I'm going to wake you up and I'm going to wake you up and I'm going to wake you up and I'm going to wake you up." He said "I'm sorry, I'm sorry, you're going to wake me up and I'm going to wake you up. I said 'you are going to wake me up and I'm going to wake you up and I'm going to wake you up and I'm going to wake you up and I'm going to wake you up and I'm going to wake you up and I'm going to wake you up and I'm going to wake you up and I'm going to wake you up and

JOKE NR 887: <|endoftext|>START:What did the Mexican say to his girlfriend after she got off the plane? "I'm so tired, my wife is so tired, I'm so tired"<|endoftext|> 

JOKE NR 888: <|endoftext|>START:A man walks into a bar. He asks the bartender, "Why the hell are you drinking?"
"I don't know," the bartender says.<|endoftext|> 

JOKE NR 889: <|endoftext|>START:My friend told me I was gay. He said, "You know what, I'm going to be gay forever."<|endoftext|> 

JOKE NR 890: <|endoftext|>START:I had a dream about a girl with no legs... She said she had no legs and couldn't walk.<|endoftext|> 

JOKE NR 891: <|endoftext|>START:So a guy walks into a bar... He says "Hey guys, how are we getting to the bar?" The guy asks him, "I'm going to the bar and I'm going to tell the bartender, 'I'm gonna be the one who gets the beer.'"

The bartender says, "Well, I'll be the one who gets the beer, and you're going to have to tell the bartender, 'I'm gonna be the one who gets the beer, and you're going to

JOKE NR 939: <|endoftext|>START:Why did the man in the gorilla suit get arrested for battery? He had a battery in his pocket.<|endoftext|> 

JOKE NR 940: <|endoftext|>START:Why are there no Jews on Mars? Because they're not there.<|endoftext|> 

JOKE NR 941: <|endoftext|>START:I like my women like I like my coffee... Light and fluffy, and full of life<|endoftext|> 

JOKE NR 942: <|endoftext|>START:A woman walks into a bar......and the bartender asks, "What's wrong?"

She says, "I don't know."<|endoftext|> 

JOKE NR 943: <|endoftext|>START:My wife is a lesbian......and I'm a gay pornstar.<|endoftext|> 

JOKE NR 944: <|endoftext|>START:You can't tell a girl's sex from her vagina. It's just not there.<|endoftext|> 

JOKE NR 945: <|endoftext|>START:What did the chicken say to the cow? "I'm not going to be a chicken anymore."<|endoftext|> 

JOKE NR 946: <|endoftext|>START:Why couldn't Hitler win the election? Because he had no balls<|endoftext|> 

JOKE NR 947: <|endoftext|>START:I used to b

JOKE NR 994: <|endoftext|>START:Why do you always get a bad joke from the opposite sex? It always gets you a little off<|endoftext|> 

JOKE NR 995: <|endoftext|>START:So I went to my local bar and ordered a beer... I was told it was a little too much.<|endoftext|> 

JOKE NR 996: <|endoftext|>START:A man is walking down the sidewalk when he sees a black man with a gun. 

"Hey man, what's that?" 

"A gun."<|endoftext|> 

JOKE NR 997: <|endoftext|>START:What do you call a man who is not a feminist? A feminist<|endoftext|> 

JOKE NR 998: <|endoftext|>START:What is the difference between a man and a dog? A dog can walk on a leash.<|endoftext|> 

JOKE NR 999: <|endoftext|>START:If someone is a pedophile, why do they call him pedophile-in-law? Because he's a pedophile-in-law.<|endoftext|> 

